In [1]:
import os
from oss2 import SizedFileAdapter, determine_part_size
from oss2.models import PartInfo
import oss2

def uploadFileOSS(fname,local_fpath):
    # 阿里云主账号AccessKey拥有所有API的访问权限，风险很高。强烈建议您创建并使用RAM账号进行API访问或日常运维，请登录 https://ram.console.aliyun.com 创建RAM账号。
    auth = oss2.Auth('LTAIj6Mqlb0Jbqo5', 'q8oDj6FNA8b6HuVeMMWCVjgB4XFAtW')
    # Endpoint以杭州为例，其它Region请按实际情况填写。
    bucket = oss2.Bucket(auth, 'http://oss-cn-hangzhou.aliyuncs.com', 'eyisheng-hangzhou')

    key = 'shared/'+fname
    filename = local_fpath

    total_size = os.path.getsize(filename)
    # determine_part_size方法用来确定分片大小。
    part_size = determine_part_size(total_size, preferred_size=100000 * 1024)

    # 初始化分片。
    # 如果需要在初始化分片时设置文件存储类型，请在init_multipart_upload中设置相关headers，参考如下。
    # headers = dict()
    # headers["x-oss-storage-class"] = "Standard"
    # upload_id = bucket.init_multipart_upload(key, headers=headers).upload_id
    upload_id = bucket.init_multipart_upload(key).upload_id
    parts = []

    # 逐个上传分片。
    with open(filename, 'rb') as fileobj:
        part_number = 1
        offset = 0
        while offset < total_size:
            print('upload',part_number,offset/total_size)
            num_to_upload = min(part_size, total_size - offset)
            # SizedFileAdapter(fileobj, size)方法会生成一个新的文件对象，重新计算起始追加位置。
            result = bucket.upload_part(key, upload_id, part_number,
                                        SizedFileAdapter(fileobj, num_to_upload))
            parts.append(PartInfo(part_number, result.etag))

            offset += num_to_upload
            part_number += 1

    # 完成分片上传。
    # 如果需要在完成分片上传时设置文件访问权限ACL，请在complete_multipart_upload函数中设置相关headers，参考如下。
    # headers = dict()
    # headers["x-oss-object-acl"] = oss2.OBJECT_ACL_PRIVATE
    # bucket.complete_multipart_upload(key, upload_id, parts, headers=headers)
    bucket.complete_multipart_upload(key, upload_id, parts)

    # 验证分片上传。
    with open(filename, 'rb') as fileobj:
        assert bucket.get_object(key).read() == fileobj.read()
        
    return


def downloadFileOSS(fname,local_fpath):
    # 阿里云主账号AccessKey拥有所有API的访问权限，风险很高。强烈建议您创建并使用RAM账号进行API访问或日常运维，请登录RAM控制台创建RAM账号。
    auth = oss2.Auth('LTAIj6Mqlb0Jbqo5', 'q8oDj6FNA8b6HuVeMMWCVjgB4XFAtW')
    # Endpoint以杭州为例，其它Region请按实际情况填写。
    bucket = oss2.Bucket(auth, 'http://oss-cn-hangzhou.aliyuncs.com', 'eyisheng-hangzhou')

    # 下载OSS文件到本地文件。如果指定的本地文件存在会覆盖，不存在则新建。
    #  <yourLocalFile>由本地文件路径加文件名包括后缀组成，例如/users/local/myfile.txt。
    bucket.get_object_to_file(os.path.join('shared/',fname), local_fpath)
    return

def listFileOSS():
    # 阿里云主账号AccessKey拥有所有API的访问权限，风险很高。强烈建议您创建并使用RAM账号进行API访问或日常运维，请登录RAM控制台创建RAM账号。
    auth = oss2.Auth('LTAIj6Mqlb0Jbqo5', 'q8oDj6FNA8b6HuVeMMWCVjgB4XFAtW')
    # Endpoint以杭州为例，其它Region请按实际情况填写。
    bucket = oss2.Bucket(auth, 'http://oss-cn-hangzhou.aliyuncs.com', 'eyisheng-hangzhou')

    # 列举存储空间下所有文件。
    for obj in oss2.ObjectIterator(bucket,prefix='shared/'):
        print(obj.key)
        
    return

In [9]:
uploadFileOSS(fname='metallic1-1.zip',local_fpath=r'E:\buffer\dataset\train\1.zip')

upload 1 0.0
upload 2 0.504088924200493


In [11]:
downloadFileOSS(fname='metallic1-3.zip',local_fpath=r'E:\buffer\dataset\train\3.zip')

upload 1 0.0


In [8]:
listFileOSS()

shared/
shared/biodata.zip
shared/biodata1.zip
shared/metallic1-0.zip
shared/metallic1-2.zip
shared/metallic1-3.zip


In [3]:
import os
import shutil
import zipfile
from os.path import join, getsize

In [5]:
def zip_file(src_dir):
    zip_name = src_dir +'.zip'
    z = zipfile.ZipFile(zip_name,'w',zipfile.ZIP_DEFLATED)
    for dirpath, dirnames, filenames in os.walk(src_dir):
        fpath = dirpath.replace(src_dir,'')
        fpath = fpath and fpath + os.sep or ''
        for idx,filename in enumerate(filenames):
            z.write(os.path.join(dirpath, filename),fpath+filename)
            if idx%1000==1:
                print(idx/len(filenames))
    z.close()
    return

In [6]:
zip_file(r'E:\buffer\dataset\train\0')

5.615169943118329e-06
0.005620785113061447
0.011235955056179775
0.016851124999298105
0.02246629494241643
0.02808146488553476
0.03369663482865309
0.039311804771771416
0.044926974714889746
0.050542144658008076
0.056157314601126405
0.06177248454424473
0.06738765448736306
0.0730028244304814
0.07861799437359972
0.08423316431671804
0.08984833425983638
0.0954635042029547
0.10107867414607304
0.10669384408919136
0.11230901403230968
0.11792418397542802
0.12353935391854634
0.12915452386166468
0.13476969380478301
0.14038486374790132
0.14600003369101966
0.151615203634138
0.1572303735772563
0.16284554352037464
0.16846071346349298
0.17407588340661131
0.17969105334972962
0.18530622329284796
0.1909213932359663
0.1965365631790846
0.20215173312220294
0.20776690306532128
0.2133820730084396
0.21899724295155792
0.22461241289467626
0.2302275828377946
0.2358427527809129
0.24145792272403124
0.24707309266714958
0.2526882626102679
0.2583034325533862
0.26391860249650456
0.2695337724396229
0.27514894238274124
0.28